### Guía 2 - Ejercicio 2

Considere el entramado mostrado en la figura, con una fuerza aplicada de 20 kN. Calcule los desplazamientos de cada uno de los nodos y las tensiones que sufre cada elemento. Todos los elementos tienen E = 210 GPa y una sección de 10 cm2, excepto el elemento 3, que tiene una sección de 20 cm2. Los elementos 2 y 5 tienen una longitud de 8 metros y el elemento 3 de 4 metros

In [1]:
import numpy as np
import numpy.testing as nptest
import pdb
np.set_printoptions(precision=4, linewidth=132)
# np.set_printoptions(linewidth=100)

#### Definición de clases

Defino la clase "Node", que va a contener la información de cada nodo. 
Los parámetros que se tiene que dar van a ser
* id: Está relacionado al número de nodo que se le asigne, y por lo tanto, va a decir a qué subdirecciones se refiere.
* x: Es la posición en x del nodo, dentro del sistema global.
* y: Es la posición en y del nodo, dentro del sistema global

In [2]:
class Node:
    # Default constructor
    def __init__(self, id, x, y):
        self.id = id
        self.x = x
        self.y = y

Se define la clase "BarElement", la cual va a contener la información de cada barra. Los parámetros que necesita son:
* node1: Le asigno un elemento de clase "Node", el cual me brinda la información del primer nodo que compone la barra.
* node2: Le asigno un elemento de clase "Node", el cual me brinda la información del segundo nodo que compone la barra.
* youngModule: Le asigno el módulo de Young de la barra
* section: Le asigno la sección que tiene la barra, asumiento que es la misma en toda su extensión.

A su vez, ésta clase, define las siguientes funciones, las cuáles permiten el calculo y devuelven:
* angle: El ángulo de la barra respecto del sistema global.
*stiffMatrix: Da la matriz de rigidez del elemento
* length: El largo del elemento
* strengthConstant: La constante de rigidez del elemento, calculado como k = E.A/L

In [3]:
class BarElement:
    # Default constructor
    def __init__(self, node1, node2, youngModule, section):
        self.node1 = node1
        self.node2 = node2
        self.youngModule = youngModule
        self.section = section

    def angle(self):
        return np.arctan2(self.node2.y - self.node1.y, self.node2.x - self.node1.x)

    def stiffMatrix(self):
        angle = self.angle()
        c = np.cos(angle)
        s = np.sin(angle)

        return self.stiffConstant()*np.array([[c**2, c*s, -c**2, -c*s],
                                              [c*s, s**2, -c*s, -s**2],
                                              [-c**2, -c*s, c**2, c*s],
                                              [-c*s, -s**2, c*s, s**2]])

    def length(self):
        deltaX = self.node2.x - self.node1.x
        deltaY = self.node2.y - self.node1.y
        return np.sqrt(deltaX**2 + deltaY**2)

    def stiffConstant(self):
        return self.youngModule * self.section / self.length()

Debería conocer qué fuerzas y desplazamientos conozco, y colocarlos en un mismo vector para cada tipo de matriz. Siempre manteniendo la notación

Se define la clase "Force", la cual va a contener la información de la fuerza que se tiene como dato. Los parámetros que necesita son:
* id: Está relacionado al número de fuerza que le asigno. 
* value: Le doy el valor, según lo que tenga en el eje, ya sea positivo o negativo 
* node: Se le asigna el número de nodo al cuál está actuando
* axis: Se le asigna 'x', 'y' o 'z', según el eje donde esté actuando la fuerza
* section: Le asigno la sección que tiene la barra, asumiento que es la misma en toda su extensión.

A su vez, ésta clase, define las siguientes funciones, las cuáles permiten el calculo y devuelven:
* s(self): Me devuelve un valor, que corresponde a un "puntero" que me dice a qué posición del vector de fuerzas hace refencia

> Es necesario darle número de id? Es útil a futuro?

In [4]:
class Force:
    def __init__(self, id, value, node, axis):
        self.id = id
        self.value = value
        # Doy el número de nodo al cual pertenece
        self.node = node
        self.axis = axis

    def s(self):
        if self.axis == 'x':
            sPoint = 2 * self.node
        # Tomo que siempre va a ser y, a menos que le indique
        elif self.axis == 'y':
            sPoint = 2 * self.node + 1
        else:
            print('Me estas dando mal los datos')
        return sPoint

> NOTA:  si quiero pasarlo a tres dimensiones, tendría que ver de reescribir para que me quede el eje Z. Esperaría que puedo poner: 

* Para X: 3 * self.node
* Para Y: 3 * self.node + 1
* Para Z: 3 * self.node + 2

> El tema es que previamente tengo que haber definido que es un problema de tres variables. Debería ver cómo cambiarlo.

Se define la clase "Displacement", la cual va a contener la información de los desplazamientos que se tiene como dato. Los parámetros que necesita son:
* id: Está relacionado al número de fuerza que le asigno. 
* value: Le doy el valor, según lo que tenga en el eje, ya sea positivo o negativo 
* node: Se le asigna el número de nodo al cuál está actuando
* axis: Se le asigna 'x', 'y' o 'z', según el eje donde esté actuando la fuerza
* section: Le asigno la sección que tiene la barra, asumiento que es la misma en toda su extensión.

A su vez, ésta clase, define las siguientes funciones, las cuáles permiten el calculo y devuelven:
* r(self): Me devuelve un valor, que corresponde a un "puntero" que me dice a qué posición del vector de fuerzas hace refencia

In [5]:
class Displacement:
    # Si axis es = 1, entonces está en X
    def __init__(self, value, node, axis):
        # self.id = id
        self.value = value
        # Doy el número de nodo al cual pertenece
        self.node = node
        self.axis = axis
        self.id = self.get_id()

    def get_id(self):
        if self.axis == 'x':
            rPoint = 2 * self.node
        # Tomo que siempre va a ser y, a menos que le indique
        elif self.axis == 'y':
            rPoint = 2 * self.node + 1
        else:
            print('Me estas dando mal los datos, ponete las pilas')
        return rPoint

#### Definición de funciones

Se define la función "globalMatrixCalc(elementsArray)", la cual realiza el cálculo de la matriz global.
- Inputs:
    * elementsArray: Se introduce un array de elementos de la clase "barElement".
- Outputs:
    * globalMatrix: Devuelve la matriz global del sistema dado.

In [6]:
# La función me devuelve la matriz global
def globalMatrixCalc(elementsArray, nodeArray):
    nodeNumber = len(nodeArray)
    elementsNumber = len(elementsArray)
    # elementsNumber = elementsArray.size
    # Genero la matriz global

    globalMatrix = np.zeros((2*nodeNumber, 2*nodeNumber))

    for i in range(elementsNumber):
        elementMatrix = elementsArray[i].stiffMatrix()
        # print('Imprimo la matriz del elemento', i, '\n', elementMatrix)
        cupleA = [2*elementsArray[i].node1.id, 2*elementsArray[i].node1.id + 1]
        cupleB = [2*elementsArray[i].node2.id, 2*elementsArray[i].node2.id + 1]

        indice1 = np.array([0, 1])
        indice2 = np.array([2, 3])

        # pdb.set_trace()

        globalMatrix[np.ix_(cupleA, cupleA)] += elementMatrix[np.ix_(indice1, indice1)]
        globalMatrix[np.ix_(cupleB, cupleA)] += elementMatrix[np.ix_(indice2, indice1)]
        globalMatrix[np.ix_(cupleB, cupleB)] += elementMatrix[np.ix_(indice2, indice2)]
        globalMatrix[np.ix_(cupleA, cupleB)] += elementMatrix[np.ix_(indice1, indice2)]

    return globalMatrix

Se define la función "vectorsRS(forcesData, displacementData)", la cual genera los vec tores de indices de incognicas o de vinculos, según la información que se dé.
- Inputs:
    * dataVector: Se introduce un array de elementos de la clase "Force", en el caso de querer calcular el vector de índices de incógnitas 'r' o de la clase "Displacement", en el caso de querer calcular el vector de índices de vinculos
- Outputs:
    * dataVectorIndex: Devuelve el vector de índices, según los datos suministrados.

In [7]:
def vectorRS(dataVector):
    dataNumber = len(dataVector)
    dataVectorIndex = []

    for i in range(0, dataNumber):
        dataVectorIndex.append(dataVector[i].id)

    return dataVectorIndex

#### Sección de testeo

Se realiza el testeo de la matriz de rigidez de un elemento. Se compara el resultado con el dado en la presentación "Método de Elementos
Finitos, Ensamble de Matrices Elementales" de la materia

In [8]:
# Test Element
def stiffMatrixTest():
    n1 = Node(0, 0, 0)
    n2 = Node(1, 1, 0)
    young = 210E9  # En GPa
    # Las secciones estan en metros cuadradoS
    e1 = BarElement(n1, n2, young, 0.001)
    correct = np.array([[1, 0, -1, 0],
                        [0, 0, 0, 0],
                        [-1, 0, 1, 0],
                        [0, 0, 0, 0]])

    nptest.assert_almost_equal(e1.stiffMatrix(), e1.stiffConstant()*correct)


stiffMatrixTest()

>Acá se pretende hacer el testeo de la matriz global. Que sigue sin darme

In [9]:
# Test de GlobalMatrix
def globalMatrixTest():
    # En GPa, pero al agregar el E9 lo paso a pascales
    young = 210E9

    n1 = Node(0, 0, 0)
    n2 = Node(1, 8, 4)
    n3 = Node(2, 16, 0)
    n4 = Node(3, 8, 0)
    nodeArray = [n1, n2, n3, n4]

    # Las secciones estan en metros cuadrados.
    e1 = BarElement(n1, n2, young, 0.001)
    e2 = BarElement(n1, n4, young, 0.001)
    e3 = BarElement(n2, n4, young, 0.002)
    e4 = BarElement(n2, n3, young, 0.001)
    e5 = BarElement(n3, n4, young, 0.001)

    elementArray = [e1, e2, e3, e4, e5]
    test = np.array([[4.5033e+07,  9.3915e+06, -1.8783e+07, -9.3915e+06,  0.0000e+00,  0.0000e+00, -2.6250e+07,  0.0000e+00],
                     [9.3915e+06,  4.6957e+06, -9.3915e+06, -4.6957e+06,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
                     [-1.8783e+07, -9.3915e+06,  1.8783e+07,  9.3915e+06,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
                     [-9.3915e+06, -4.6957e+06,  9.3915e+06,  1.0970e+08,  0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0500e+08],
                     [0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
                     [0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
                     [-2.6250e+07,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  2.6250e+07,  0.0000e+00],
                     [0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0500e+08,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0500e+08]])
    # print(globalMatrixCalc(elementArray, nodeArray))
    calcMatrix = globalMatrixCalc(elementArray, nodeArray)
    print(calcMatrix)
    nptest.assert_almost_equal(calcMatrix, test, 4)


globalMatrixTest()

[[ 4.5033e+07  9.3915e+06 -1.8783e+07 -9.3915e+06  0.0000e+00  0.0000e+00 -2.6250e+07  0.0000e+00]
 [ 9.3915e+06  4.6957e+06 -9.3915e+06 -4.6957e+06  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]
 [-1.8783e+07 -9.3915e+06  3.7566e+07 -5.5879e-09 -1.8783e+07  9.3915e+06 -3.9369e-25  6.4294e-09]
 [-9.3915e+06 -4.6957e+06 -5.5879e-09  1.1439e+08  9.3915e+06 -4.6957e+06  6.4294e-09 -1.0500e+08]
 [ 0.0000e+00  0.0000e+00 -1.8783e+07  9.3915e+06  4.5033e+07 -9.3915e+06 -2.6250e+07  3.2147e-09]
 [ 0.0000e+00  0.0000e+00  9.3915e+06 -4.6957e+06 -9.3915e+06  4.6957e+06  3.2147e-09 -3.9369e-25]
 [-2.6250e+07  0.0000e+00 -3.9369e-25  6.4294e-09 -2.6250e+07  3.2147e-09  5.2500e+07 -9.6441e-09]
 [ 0.0000e+00  0.0000e+00  6.4294e-09 -1.0500e+08  3.2147e-09 -3.9369e-25 -9.6441e-09  1.0500e+08]]


AssertionError: 
Arrays are not almost equal to 4 decimals

Mismatched elements: 31 / 64 (48.4%)
Max absolute difference: 45032971.011
Max relative difference: 1.
 x: array([[ 4.5033e+07,  9.3915e+06, -1.8783e+07, -9.3915e+06,  0.0000e+00,  0.0000e+00, -2.6250e+07,  0.0000e+00],
       [ 9.3915e+06,  4.6957e+06, -9.3915e+06, -4.6957e+06,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
       [-1.8783e+07, -9.3915e+06,  3.7566e+07, -5.5879e-09, -1.8783e+07,  9.3915e+06, -3.9369e-25,  6.4294e-09],...
 y: array([[ 4.5033e+07,  9.3915e+06, -1.8783e+07, -9.3915e+06,  0.0000e+00,  0.0000e+00, -2.6250e+07,  0.0000e+00],
       [ 9.3915e+06,  4.6957e+06, -9.3915e+06, -4.6957e+06,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
       [-1.8783e+07, -9.3915e+06,  1.8783e+07,  9.3915e+06,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],...

> Cuando hago la matriz, los elementos del centro me quedan mal. Estoy indexando mal, al parecer, pero no entiendo cómo hacer que quede bien. Por eso no me pasa el test. Se puede ver en el print de la matriz.

#### Definición de nodos y elementos

In [10]:
n1 = Node(0, 0, 0)
n2 = Node(1, 8, 4)
n3 = Node(2, 16, 0)
n4 = Node(3, 8, 0)
nodeArray = [n1, n2, n3, n4]

In [11]:
young = 210E9  # En Pa
# Las secciones estan en metros cuadrados.
e1 = BarElement(n1, n2, young, 0.001)
e2 = BarElement(n1, n4, young, 0.001)
e3 = BarElement(n2, n4, young, 0.002)
e4 = BarElement(n2, n3, young, 0.001)
e5 = BarElement(n3, n4, young, 0.001)

elementsArray = [e1, e2, e3, e4, e5]

> Debería hacer algún for para que me arme directamente el array de elementos. Lo mismo con las fuerzas y los desplazamientos

In [12]:
# Calculo la matrix global
globalMatrix = globalMatrixCalc(elementsArray, nodeArray)

In [21]:
# Datos de fuerzas y desplazamientos que conozco para el ejercicio 2
# Puse el value de las fuerzas en NEWTONS
f2 = Force(2, 0, 1, 'x')
f3 = Force(3, 0, 1, 'y')
f4 = Force(4, 0, 2, 'x')
f6 = Force(6, -20000, 3, 'y')
f7 = Force(7, 0, 3, 'x')
# forcesData = [f2.value, f3.value, f4.value, f6.value, f7.value]
forcesData = [f2, f3, f4, f6, f7]

# Desplazamientos
u0 = Displacement(0, 0, 'x')
u1 = Displacement(0, 0, 'y')
u5 = Displacement(0, 2, 'y')
displacementData = [u0, u1, u5]

> A comprobar: Si va con las fuerzas en Newton o en otra escala. Sí, van en Newton

In [22]:
rVector = vectorRS(forcesData)
sVector = vectorRS(displacementData)
# sVector

[0, 1, 5]

In [23]:
# Colocando 2*cantidad de nodos, me aseguro que me funcione siempre que tenga 2D.
U = np.zeros((2*len(nodeArray), 1))
F = np.zeros((2*len(nodeArray), 1))

# Acá tengo una forma de ver todos los .value de un array de clases.
U[sVector] = [[d.value] for d in displacementData]
F[rVector] = [[d.value] for d in forcesData]

In [24]:
# Cálculo de los desplazamientos y fuerzas
auxiliarF = F[rVector] - globalMatrix[np.ix_(rVector, sVector)].dot(U[sVector])
U[rVector] = np.linalg.solve(globalMatrix[np.ix_(rVector, rVector)], auxiliarF)
# print(F[sVector])
# print(globalMatrix[sVector, :])
# print(U)
F[sVector] = globalMatrix[sVector, :].dot(U)

[[0.]
 [0.]
 [0.]]
[[ 4.5033e+07  9.3915e+06 -1.8783e+07 -9.3915e+06  0.0000e+00  0.0000e+00 -2.6250e+07  0.0000e+00]
 [ 9.3915e+06  4.6957e+06 -9.3915e+06 -4.6957e+06  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  9.3915e+06 -4.6957e+06 -9.3915e+06  4.6957e+06  3.2147e-09 -3.9369e-25]]
[[ 0.    ]
 [ 0.    ]
 [-0.0004]
 [ 0.0008]
 [-0.0008]
 [ 0.    ]
 [-0.0008]
 [ 0.0008]]
